In [2]:
def calculate_State(gdf_3857,buildingsData,para):
    
    
    user_buffer    = para.Effective_range   # metre
    building_level = para.Level_avg_height  # metre
    
    gdf = gdf_3857
    
    for index, row in gdf.iterrows():
        
        if row['SAT_EL'] <= 10 :
            
            gdf.at[index, 'State'] = 'out_of_range'
        
        else :
            
            # get user loc
            # ========================================
            user_loc_x = row['geometry'].coords[0][0]
            user_loc_y = row['geometry'].coords[0][1]
            #print(user_loc_x, user_loc_y)
            
            clear_output(wait=True)
            print('caluculate state')
            print('running : ',index,'/',gdf.shape[0])
            
            end_loc_x = user_loc_x + user_buffer * math.cos(math.radians(90-row['SAT_AZ']))
            end_loc_y = user_loc_y + user_buffer * math.sin(math.radians(90-row['SAT_AZ']))
            
            # create linestring
            # ========================================
            user_point = Point( user_loc_x, user_loc_y, 0 )
            end_point  = Point(  end_loc_x,  end_loc_y, 0 ) 
            user_end_string = LineString([user_point, end_point])
            
            # gdf string
            # ========================================
            gdf_string = gpd.GeoDataFrame(columns=['name','geometry'],crs='3857')
            gdf_string['geometry'] = None
            gdf_string.at[1, 'name'] = 'string'
            gdf_string.at[1, 'geometry'] = user_end_string
            gdf_string.at[2, 'name'] = 'user_point'
            gdf_string.at[2, 'geometry'] = user_point
            gdf_string.at[3, 'name'] = 'sat_point'
            gdf_string.at[3, 'geometry'] = end_point
            gdf_string.at[4, 'name'] = 'user_buffer'
            gdf_string.at[4, 'geometry'] = user_point.buffer(user_buffer)
            
            gdf_buf = gdf_string[gdf_string['name']=='user_buffer']
            gdf_str = gdf_string[gdf_string['name']!='user_buffer']
            
            buildings_in_buffer =  gpd.sjoin(buildingsData,gdf_buf,how='inner', op='intersects')
            buildings_in_buffer = buildings_in_buffer.drop(['index_right'], axis=1)
            
            buildings_in_string =  gpd.sjoin(buildings_in_buffer,gdf_str,how='inner', op='intersects')
            


            temp_state = 'LOS'
            x0 = user_loc_x
            y0 = user_loc_y
            
            if (buildings_in_string.empty != True) :
                
                intersection_strings = gdf_string[gdf_string['name']=='string'].intersection(buildings_in_string.unary_union)
                intersection_strings_explode  = intersection_strings.explode(ignore_index=True)
                
                for k in range(0,len(intersection_strings_explode)) :
                    
                    t = intersection_strings_explode[k]
                    #print(t)
                    x1 = t.coords[0][0]
                    y1 = t.coords[0][1]
                    z1 = t.coords[0][2]

                    x2 = t.coords[1][0]
                    y2 = t.coords[1][1]
                    z2 = t.coords[1][2]

                    d1 = ((x1-x0)**2+(y1-y0)**2)**(1/2)
                    d2 = ((x2-x0)**2+(y2-y0)**2)**(1/2)

                    z_max = max(z1,z2)
                    
                    check_los_d1 = d1*math.tan(math.radians(row['SAT_EL']))/3.5
                    if(check_los_d1 <= z_max) :
                        temp_state = 'NLOS'
                        
                    check_los_d2 = d2*math.tan(math.radians(row['SAT_EL']))/3.5
                    if(check_los_d2 <= z_max) :
                        temp_state = 'NLOS'
            
            gdf.at[index, 'State'] = temp_state
    
    return gdf